In [1]:
import torchvision
from research_task_utils_for_autosearch import *
from models_autosearch import *
from torchsummary import summary
import numpy as np
import json

In [2]:
batchsize = 512
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor()
    ]
)
traindata = torchvision.datasets.CIFAR100(root="../dataset", train=True, download=True, transform=transform)
testdata = torchvision.datasets.CIFAR100(root="../dataset", train=False, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(traindata, batch_size=batchsize, shuffle=True)
testloader = torch.utils.data.DataLoader(testdata, batch_size=batchsize, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# architecture_params = Architecture_params(k1=1, c_out1=32, c_inner1=128,
#                                          k2=1, c_out2=32, c_inner2=128,
#                                          k3=1, c_out3=64, c_inner3=128,
#                                         k4=1, c_out4=100, c_inner4=256)
# model = Model(architecture_params)
# model = model.to(device)
# optimizer = torch.optim.Adam(model.parameters())
# loss_function = torch.nn.CrossEntropyLoss()
# statistics_dict = init_statistics(model, device, trainloader, testloader, loss_function)

In [4]:
# statistics_dict

In [5]:
# num_of_params(model)

## Подпстранство для поиска архитектур ("Space_1")

Поищем более хороший набор количества каналов для архитектуры из 4-х MBConv слоев

При этом положим c_inner{i}=2*c_out{i}

Т.е. пространство четырехмерное, координаты:
q{i} = c_out{i}

In [3]:
def generate_architecture_params(q1, q2, q3, q4):
    return Architecture_params(k1=1, c_out1=q1, c_inner1=2*q1,
                               k2=1, c_out2=q2, c_inner2=2*q2,
                               k3=1, c_out3=q3, c_inner3=2*q3,
                               k4=1, c_out4=q4, c_inner4=2*q4)


# Функция для пля обучения одной модели и сохранения всех логов и чекпоинтов в файлы
def perform_experiment(q, epochs_n=80, name=""):
    q1, q2, q3, q4 = q
    architecture_params = generate_architecture_params(q1, q2, q3, q4)
    model = Model(architecture_params)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters())
    loss_function = torch.nn.CrossEntropyLoss()
    statistics_dict = init_statistics(model, device, trainloader, testloader, loss_function)
    train(model, device, optimizer, loss_function, trainloader, testloader, statistics_dict, epochs_n=epochs_n,
          augmenter=None,
          checkpoints_dir=f"../tmp_checkpoints/architecture_search_v2/checkpoints/space_1/q{q1}_{q2}_{q3}_{q4}_"+name,
          checkpoints_per=25)
    add_statistics_to_json(model_info={"name": "Space_1"+name, "configuration": q,
                                       "comment": "", "num_of_params": num_of_params(model),
                                       "acc": statistics_dict["testacc"][-1]},
                           statistics_dict=statistics_dict,
                           path_dir="../tmp_checkpoints/architecture_search_v2/statistics",
                           filename="Space_1_statistics.json")
    print(f"Обучение для {q} завершено, testacc=",statistics_dict["testacc"][-1])

In [19]:
perform_experiment(q=(5,5,5,5), epochs_n=1, name="func_test")

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:11<00:00, 11.42s/it]


In [4]:

# Страшная функция, которая из полных логов вытаскивает только гиперпараметры модели, кол-во параметров и accuracy.
def get_short_data(json_statistics_path="../tmp_checkpoints/architecture_search_v2/statistics/Space_1_statistics.json",
                   exp_name=""):
    short_data = dict()
    with open(json_statistics_path) as f:
        json_data = json.load(f)
        for log in json_data:
            if log["model_info"]["name"] == "Space_1"+exp_name:
                short_data[tuple(log["model_info"]["configuration"])] = {"N": log["model_info"]["num_of_params"],
                                                                  "acc": log["model_info"]["acc"]}
    return short_data
        

In [5]:
def get_grads(q: tuple, short_data):
    grad_N = []
    grad_acc = []
    for i in range(len(q)):
        q2 = list(q)
        q2[i] += 1
        q2 = tuple(q2)
        grad_N.append(short_data[q2]["N"] - short_data[q]["N"])
        grad_acc.append(short_data[q2]["acc"] - short_data[q]["acc"])
    return np.array(grad_N), np.array(grad_acc)

In [6]:
q1, q2, q3, q4 = 32, 32, 64, 128
architecture_params = generate_architecture_params(q1, q2, q3, q4)
model = Model(architecture_params)
print(num_of_params(model))

99684


In [29]:
tuple(np.array((3, 4)))

(3, 4)

In [49]:
# Поиск архитектуры

name = "test1"
# q = (5,5,5,5)
q = (32, 32, 64, 128)
K = 5.
epochs_n = 75


story = []
while True:
    print(f"q = {q}, начало измерений в окрестности q")
    perform_experiment(q, epochs_n=epochs_n, name=name)
    for i in range(len(q)):
        q2 = list(q)
        q2[i] += 1
        q2 = tuple(q2)
        perform_experiment(q2, epochs_n=epochs_n, name=name)
    short_data = get_short_data(exp_name=name)
    grad_N, grad_acc = get_grads(q, short_data)
    
    story.append((q, short_data[q]["N"], short_data[q]["acc"]))# сохраняем историю изменения моделей
    
    q_prev = tuple(q) # нужно только для принтов
    
    direction_to_change_params = grad_acc - np.dot(grad_N, grad_acc) / np.dot(grad_N, grad_N) * grad_N
    direction_to_change_params /= np.linalg.norm(direction_to_change_params)
        
    new_hyperparams = np.array(q) + K*direction_to_change_params
    new_hyperparams = np.rint(new_hyperparams).astype(int)
    
    
    # !!! WARNING, поменять это на другое условие, если буду работать с k{i} -- в другом пр-ве!
    for i in range(len(new_hyperparams)):
        if new_hyperparams[i] < 1:
            new_hyperparams[i] = 1
            
    q = tuple(new_hyperparams.tolist())
    
    
    print("Конец измерений в окрестности q, some results:")
    print("acc(q): ", short_data[q_prev]["acc"])
    
    print(short_data)
    print("grad N:")
    print(grad_N)
    print("grad acc:")
    print(grad_acc)
    print("Новое значение q:", q)
    print("Предсказываемые значения N и acc для нового q: ",
          int(short_data[q_prev]["N"] + np.dot(grad_N, K*direction_to_change_params)),
          short_data[q_prev]["acc"] + np.dot(grad_acc, K*direction_to_change_params))

q = (32, 32, 64, 128), начало измерений в окрестности q


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [35:53<00:00, 28.72s/it]


Обучение для q=(32, 32, 64, 128) завершено, testacc= 0.3205


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [41:07<00:00, 32.90s/it]


Обучение для q=(33, 32, 64, 128) завершено, testacc= 0.3256


...

Обучение для q=(30, 28, 67, 130) завершено, testacc= 0.3238


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [39:05<00:00, 31.27s/it]


Обучение для q=(31, 28, 67, 130) завершено, testacc= 0.3144


  0%|                                                                                                                                | 0/75 [00:07<?, ?it/s]


KeyboardInterrupt: 

In [7]:
# Эксперимент для улучшения метода оценки acc.
# Проводим обучение одинаковых моделей 10 раз, набираем статистику
# основная цель -- оценить дисперсию acc.

name = "10_times_training"
q = (32, 32, 64, 128)
epochs_n = 100


for k in range(10):
    perform_experiment(q, epochs_n=epochs_n, name=name)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [45:48<00:00, 27.48s/it]


Обучение для (32, 32, 64, 128) завершено, testacc= 0.3346


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [45:44<00:00, 27.45s/it]


Обучение для (32, 32, 64, 128) завершено, testacc= 0.3336


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [45:44<00:00, 27.44s/it]


...

Обучение для (32, 32, 64, 128) завершено, testacc= 0.3236


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [45:53<00:00, 27.53s/it]


Обучение для (32, 32, 64, 128) завершено, testacc= 0.3259


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [45:56<00:00, 27.57s/it]

Обучение для (32, 32, 64, 128) завершено, testacc= 0.3375


In [6]:
# Эксперимент для оценки изменения acc вдоль координатных направлений

q1_init = 32
q2_init = 32
q3_init = 64
q4_init = 128

name = "q1_direction_tests"
epochs_n = 75

for q1 in [q1_init, q1_init - 1, q1_init + 1, q1_init - 3, q1_init + 3, q1_init - 10, q1_init + 10]:
    perform_experiment((q1, q2_init, q3_init, q4_init), epochs_n=epochs_n, name=name)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [34:10<00:00, 27.34s/it]


Обучение для (32, 32, 64, 128) завершено, testacc= 0.3043


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [34:04<00:00, 27.27s/it]


Обучение для (31, 32, 64, 128) завершено, testacc= 0.3251


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [35:59<00:00, 28.79s/it]


...

Обучение для (35, 32, 64, 128) завершено, testacc= 0.333


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [30:00<00:00, 24.01s/it]


Обучение для (22, 32, 64, 128) завершено, testacc= 0.3097


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [41:39<00:00, 33.32s/it]

Обучение для (42, 32, 64, 128) завершено, testacc= 0.3345
